In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
import os
from mpl_axes_aligner import align
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import norm
from scipy.stats import multivariate_normal
import h5py
import math
import torch
from torchviz import make_dot
from JPwaptool import JPwaptool
import wf_func as wff

In [2]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

In [3]:
xn = np.linspace(-5, 5, 600)
yn = np.linspace(-5, 5, 600)
xv, yv = np.meshgrid(xn, yn)
pos = np.dstack((xv, yv))

m = np.array([[-1., 1.5], [-1., 1.]])
s = np.array([[1., .5], [.6, .8]])
a = np.array([[.4, .6], [.4, .6]])
rho = np.array([[0.5, -0.7], [0.4, 0.4]])

ymax = max(np.max(normcombine(xn, m[0, :], s[0, :], a[0, :])), np.max(normcombine(yn, m[1, :], s[1, :], a[1, :])))
fig = plt.figure(figsize=(5, 5))
ax0 = fig.add_axes((.1, .85, .75, .12))
ax0.axis('off')
ax0.plot(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), color='b')
ax0.fill_between(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), facecolor='blue', alpha=0.5, color=None)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(0, ymax)
ax1 = fig.add_axes((.85, .1, .12, .75))
rot = transforms.Affine2D().rotate_deg(-90)
base = plt.gca().transData
ax1.axis('off')
ax1.plot(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), color='b', transform=rot+base)
ax1.fill_between(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), facecolor='blue', alpha=0.5, color=None, transform=rot+base)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlim(0, ymax)
ax2 = fig.add_axes((.1, .1, .75, .75))
ax2.contourf(xv, yv, normcombine2d(pos, m, s, a, rho), cmap=cm.PuBu)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$\mu$')
ax2.set_ylabel(r'$\nu$')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
fig.savefig('Note/figures/wdistdemo.pgf')
fig.clf()
plt.close(fig)

In [4]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(2, 2, figure=fig, left=0.12, right=0.95, top=0.97, bottom=0.15, wspace=0.4, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$weight\ a$')
ax0.set_xticks(t)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$weight\ b_{1}$')
ax1.set_xticks(t)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$weight\ b_{2}$')
ax2.set_xticks(t)
ax3.text(0, 0, r'$L_{2}\ =\ 0.25$')
ax3.set_xlim(0, 5)
ax3.set_ylim(-1, 1)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
fig.savefig('Note/figures/tab.pgf')
fig.clf()
plt.close(fig)

In [8]:
stream = JPwaptool(1029, 150, 600, 7, 15)
spe_pre = wff.read_model('spe.h5')
with h5py.File('/srv/waveform-analysis/originztraining/ztraining-1.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Waveform'][:]
    tru = ipt['GroundTruth'][:]

In [10]:
i = 4
stream.Calculate(ent[i]['Waveform'])
cid = ent[i]['ChannelID']
eid = ent[i]['EventID']
truth = np.sort(tru[(tru['EventID'] == eid) & (tru['ChannelID'] == cid)], kind='stable', order=['EventID', 'ChannelID', 'RiseTime'])
wave = (ent[i]['Waveform'] - stream.ChannelInfo.Pedestal) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'RiseTime':'HitTime'})
df = df[:13]
charge = df['Charge'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'EventID' : 'str', 'ChannelID' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['Charge'] = ['{:.02f}'.format(s) for s in charge]

In [11]:
df

,EventID,ChannelID,HitTime,Charge
0,1,4,271,256.23
1,1,4,272,70.78
2,1,4,276,150.14
3,1,4,278,148.26
4,1,4,283,262.02
5,1,4,284,90.74
6,1,4,285,166.08
7,1,4,287,90.57
8,1,4,290,52.15
9,1,4,290,102.82


In [38]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['RiseTime'].min(), 0, wave.sum(), color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
rise = truth['RiseTime'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise}\mathrm{{ns}}, {vsum:.{6}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['RiseTime'].min(), wave.sum()/2), xytext=(truth['RiseTime'].min()+50, wave.sum()*1/3), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [10]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.clf()
plt.close(fig)

fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['RiseTime'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.clf()
plt.close(fig)

In [11]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['RiseTime'], 0, truth['Charge'], color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.clf()
plt.close(fig)

In [12]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum())

2363.878048780505
2388.0511151143314


In [13]:
t = np.load('/srv/waveform-analysis/dataset/jinp/Charge/ztraining-Channel00/testing_record_2020-10-07_10:30:48.npz')['arr_0']
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(1, len(t)+1), t, label='W-distance', color='C1')
ax.set_xlabel(r'$epoch$')
ax.set_ylabel(r'$Wasserstein\ Distance/\mathrm{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.clf()
plt.close(fig)

In [14]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
ax2.annotate('', xy=(pet.mean(), pwe.max()*0.7), xytext=(pet.mean()+pet.ptp()/2, pwe.max()*0.7), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829  52.15494767 102.81575556
 224.36935435  78.89552354 138.39822323 108.31198849 112.6873069
 208.81511771 126.79288372]
truth Resi-norm = 34.67568500875397
RiseTime = [269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286
 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304
 305 306 307 308 309 315 316 317 318 331 332 372 373 374 375 376 489 490], Weight = [18.95732542 37.60334248 50.42247921 53.91860741 49.25710314 46.92635101
 48.09172707 55.08398347 58.58011167 57.41473561 52.75323134 48.09172707
 51.58785527 66.73774414 85.3837612  97.03752186 97.03752186 91.21064153
 78.3915048  70.23387233 69.06849627 67.9031202  69.06849627 72.56462447
 72.56462447 62.07623987 49.25710314 42.26484674 30.61108608 27.11495788
 24.78420575 25.94958182 

In [15]:
pet = np.array(stream.ChannelInfo.PeakLoc) - spe_pre[cid]['peak_c']
pwe = np.array(stream.ChannelInfo.PeakAmp) / spe_pre[cid]['spe'].max()
pwe = pwe[pet >= 0]; pet = pet[pet >= 0]
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
loc = np.array(stream.ChannelInfo.PeakLoc)
amp = np.array(stream.ChannelInfo.PeakAmp)
for i in range(len(np.array(stream.ChannelInfo.PeakLoc))):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+35), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829  52.15494767 102.81575556
 224.36935435  78.89552354 138.39822323 108.31198849 112.6873069
 208.81511771 126.79288372]
truth Resi-norm = 34.67568500875397
RiseTime = [272 284 306 315 331 374 388 489], Weight = [447.98548992 806.24118206 273.69893698 176.87307424 157.50790169
 293.06410953  41.3168664  167.19048797]
wdist = 11.293074060960047,cdiff = -24.173066333826227
Resi-norm = 149.3495076787619


In [16]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829  52.15494767 102.81575556
 224.36935435  78.89552354 138.39822323 108.31198849 112.6873069
 208.81511771 126.79288372]
truth Resi-norm = 34.67568500875397
RiseTime = [263 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 290 291 292 293 294 299 300 301 304 305 306 307 308 314 315 316
 317 318 330 331 332 333 371 372 373 374 375 487 488 489 490 491], Weight = [ 17.87981324  49.37491829  77.67932239  82.67739575  66.42072652
  44.14380643  32.99201426  39.52852609  55.33689509  64.17979594
  55.4974312   33.99024574  17.57996542  24.55030066  58.96144998
 105.28245402 136.96339069 133.68606121  95.49249233  43.74948333
  36.43516424  74.65678088  94.77899494  83.41387982  47.59724169
  19.74169229  23.42425877  15.75534477  18.52615194  34.47816491
  

In [17]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(spe_pre[cid]['thres'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829  52.15494767 102.81575556
 224.36935435  78.89552354 138.39822323 108.31198849 112.6873069
 208.81511771 126.79288372]
truth Resi-norm = 34.67568500875397
RiseTime = [270 271 272 273 274 275 276 277 278 279 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 300 301 305 306 307 315 316 317 330 331 332 372
 373 374 488 489 490], Weight = [ 38.55964529 177.88000132  90.91102049  27.65453963  18.06879615
  28.42976381  66.5428219   95.2371267   65.34016843  26.93065986
  18.74225878  48.31602279 130.09556309 191.67414392 134.53018119
  69.09106001  36.33811299  26.94308055  28.87623916  44.95608815
  70.80651038  99.51246755  82.63383538  41.99519941  17.62542814
  22.42639323  35.99805629  62.06284148  25.65745046  26.40304765
  58.65308296  23.332021    21.69098547  49.71

In [18]:
pet, pwe0 = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe0 = wff.clip(pet, pwe0, 0.1)
pwe = pwe0 * spe_pre[cid]['spe'].sum()
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid, mode='Charge')

PEnum is 17
truth RiseTime = [271 272 276 278 283 284 285 287 290 290 293 301 306 316 331 373 489], Weight = [256.22730803  70.78286561 150.13576328 148.26017169 262.0190966
  90.73764185 166.07706861  90.57009829  52.15494767 102.81575556
 224.36935435  78.89552354 138.39822323 108.31198849 112.6873069
 208.81511771 126.79288372]
truth Resi-norm = 34.67568500875397
RiseTime = [271 272 276 277 278 280 283 284 285 286 287 289 290 293 301 306 316 331
 373 489], Weight = [257.09996851  59.97559142 107.96808676  85.1044563   81.23801201
  26.62100132 206.9286041  199.38175303  33.81030916 123.52335426
  25.63699709  20.85253084 137.69604671 223.22042708  67.06982638
 132.06522011  96.13529999  88.0440534  205.4031407  100.36041204]
wdist = 2.2689054044274855,cdiff = -109.9160239323005
Resi-norm = 36.8826943998537


In [19]:
s = [r'$\mathrm{FindPeak}$', r'$\mathrm{Threshold}$', r'$\mathrm{FFT}$', r'$\mathrm{LucyDDM}$', r'$\mathrm{CNN}$', r'$\mathrm{Fitting}$']
x = np.arange(0, 6)
y = np.array([10.71, 3.67, 1.45, 0.93, 0.248, 0.504])
dy = np.array([[1.32, 1.80, 0.86, 0.49, 0.065, 0.16], [24.06, 6.11, 2.12, 1.41, 0.425, 0.87]])
dy[0, :] = y - dy[0, :]; dy[1, :] = dy[1, :] - y
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax.bar(x, y, color='b')
ax.set_ylim(0, 26)
ax.set_ylabel(r'$\mathrm{Wasserstein}\ \mathrm{Distance}/\mathrm{ns}$')
ax.set_xticks(x)
ax.set_xticklabels(s)
ax.errorbar(x, y, yerr=dy, fmt='o', ecolor='r', color='b', elinewidth=2)
fig.savefig('Note/figures/summarycharge.pgf')
fig.clf()
plt.close(fig)

In [20]:
s = [r'$\mathrm{FindPeak}$', r'$\mathrm{Threshold}$', r'$\mathrm{FFT}$', r'$\mathrm{LucyDDM}$', r'$\mathrm{CNN}$', r'$\mathrm{Fitting}$']
x = np.arange(0, 6)
y = np.array([11.62, 4.69, 3.08, 2.74, 0.76, 2.46])
dy = np.array([[2.13, 1.78, 1.04, 0.77, 0.014, 0.57], [25.16, 8.51, 5.65, 5.24, 1.78, 4.83]])
dy[0, :] = y - dy[0, :]; dy[1, :] = dy[1, :] - y
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax.bar(x, y, color='b')
ax.set_ylim(0, 26)
ax.set_ylabel(r'$\mathrm{Wasserstein}\ \mathrm{Distance}/\mathrm{ns}$')
ax.set_xticks(x)
ax.set_xticklabels(s)
ax.errorbar(x, y, yerr=dy, fmt='o', ecolor='r', color='b', elinewidth=2)
fig.savefig('Note/figures/summarypenum.pgf')
fig.clf()
plt.close(fig)

In [21]:
with h5py.File('spe-jinp.h5', 'r', libver='latest', swmr=True) as speFile:
    s = speFile['SinglePE'][:]
    a = s[s['ChannelID'] == 0]['speWf']
    a = a[:min(10, len(a))]

In [22]:
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax.plot(a.T*-1)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
fig.savefig('Note/figures/spewaves.pgf')
fig.clf()
plt.close(fig)

In [23]:
with h5py.File('lucyddm-Charge/dist-jinp/distr-0.h5', 'r', libver='latest', swmr=True) as distfile:
    dt = distfile['Record'][:]
    
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
c, t = np.unique(dt['TotalPEpos'], return_counts=True)
ax.bar(c, t)
ax.hist(dt['TotalPEpos'], bins=np.max(dt['TotalPEpos']), histtype='step')
ax.set_yscale('log', nonpositive='mask')
ax.grid()
ax.set_xlabel(r'$N_{pos}$')
ax.set_ylabel(r'$Count$')
fig.savefig('Note/figures/pepos.pgf')
plt.close(fig)

In [24]:
methods = ['xiaopeip', 'takara']

for m in methods:
    with h5py.File(m + '-Charge/dist-jinp/distr-0.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['TotalPEpos'])
    l = min(50, penum.max())
    wdist_stats = np.zeros((l, 6))
    edist_stats = np.zeros((l, 6))
    for i in range(l):
        dtwpi = dt['wdist'][dt['TotalPEpos'] == i+1]
        dtepi = dt['RSS'][dt['TotalPEpos'] == i+1]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 10)
        wdist_stats[i, 5] = np.percentile(dtwpi, 90)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 10)
        edist_stats[i, 5] = np.percentile(dtepi, 90)

    n = 4
    dtn = dt[dt['TotalPEpos'] == n]
    a = (dtn['wdist'] < N).sum()
    b = (dtn['RSS'] < M).sum()
    L = len(dtn['wdist'])
    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
    ax.hist(dtn['wdist'][dtn['wdist']<N], bins=50, density=1)
    ax.set_xlabel(r'$W-dist/\mathrm{ns}$')
    ax.set_ylabel(r'$Normalized\ Count$')
    # ax.set_title('count {}(Wd<{:.2f}ns)/{}={:.2f}'.format(a, N, L, a/L))
    fig.savefig('Note/figures/' + m + 'chargehist.pgf')
    plt.close(fig)

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
    ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
    ax.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{90\%}_{10\%}$')
    ax.set_xlim(-1, l)
    ax.set_xlabel(r'$N_{pos}$')
    ax.set_ylabel(r'$W-dist/\mathrm{ns}$')
    # ax.set_title('W-dist vs TotalPEpos stats')
    ax.legend()
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    plt.close(fig)